# COMSOC

Welcome! This notebook will guide you through some of the functionalities of the `COMSOC` package.

## Scenarios

To begin, let us import a _theory_. A theory is a general framework modelling a family of collective decisions. For example, Voting Theory, Participatory Budgeting or Multiwinner Voting. Let us import the module implementing the theory of anonymous voting.

In [1]:
import COMSOC.anonymous as theory

Now we can focus on a specific instance, or _scenario_, of anonymous voting. Suppose that we want to model all situations where (up to) two anonymous voters need to choose between three alternatives, called `a`, `b` and `c`. To define this scenario, we can do the following:

In [2]:
nVoters = 2
alternatives = {'a', 'b', 'c'}

scenario = theory.Scenario(nVoters, alternatives)
print(scenario)

Anonymous voting scenario, with 2 voters and alternatives {a, b, c}.


Next, we can focus on a specific _preference profile_, and compute the outcome selected by some common voting rules; the Borda and Plurality rules for example.

In [3]:
profile = scenario.get_profile('a>b>c,b>a>c')

print(profile)

borda_rule = theory.rules.Borda(scenario)
plurality_rule = theory.rules.Plurality(scenario)

print(f'Borda( {profile} ) ==> {borda_rule(profile)}')
print(f'Plurality( {profile} ) ==> {plurality_rule(profile)}')

#1:a>b>c, #1:b>a>c
Borda( #1:a>b>c, #1:b>a>c ) ==> {a, b}
Plurality( #1:a>b>c, #1:b>a>c ) ==> {a, b}


Another thing we can do is to loop over all possible profiles in this scenario, and select those with a Condorcet winner. Here, we print the first 5 such cases found.

In [4]:
SetOfCW = {profile for profile in scenario.profiles if profile.hasCondorcetWinner()}

for profile in list(SetOfCW)[:5]:
    print(f"Profile {profile} has a Condorcet winner: {profile.condorcetWinner()}.")

Profile #1:b>a>c has a Condorcet winner: b.
Profile #1:a>c>b has a Condorcet winner: a.
Profile #2:b>a>c has a Condorcet winner: b.
Profile #2:c>a>b has a Condorcet winner: c.
Profile #2:c>b>a has a Condorcet winner: c.


Indeed, as we do not allow for more than 2 voters, only unanimous profiles such as those listed above can have a Condorcet winner.

Yet another thing we can do is to print all profiles with exactly one voter:

In [6]:
for profile in scenario.profilesOfSize(1):
    print(profile)

#1:a>c>b
#1:a>b>c
#1:c>a>b
#1:b>c>a
#1:b>a>c
#1:c>b>a


## Reasoning

Next, we are going to demonstrate how to perform automated reasoning tasks using this package. The module `COMSOC.problems` allows to perform various reasoning tasks regarding voting rules. Each of them offers a `.solve()` method, accepting parameters that control the strategy to be used.

A first task one can perform is to check the satisfiability of a set of axioms. Given a corpus of axioms, we are able to search for and obtain voting rules satisfying them.
First of all, let us create a corpus of axioms.

In [7]:
corpus = {
    theory.axioms.Faithfulness(scenario),
    theory.axioms.Reinforcement(scenario),
    theory.axioms.Cancellation(scenario),
    theory.axioms.Pareto(scenario),
    theory.axioms.Neutrality(scenario),
}

Is this set of axioms satisfiable? To check this, we define a new instance of `CheckAxioms`, and solve it using a SAT solver.

In [8]:
from COMSOC.problems import CheckAxioms

problem = CheckAxioms(corpus)
problem.solve(strategy = "SAT")

True

Next, we can check whether certain known voting rules satisfy our set of axioms. To this end, the problem `CheckRule` can be used.

In [9]:
from COMSOC.problems import CheckRule

problem = CheckRule(corpus, borda_rule)
print(problem.solve(strategy = "SAT"))

problem = CheckRule(corpus, plurality_rule)
print(problem.solve(strategy = "SAT"))

True
False


As we can see, the Borda rule satisfies our corpus. However, Plurality does not (as it violates Cancellation). Let us try again without this axiom.

In [10]:
corpus_small = {
    theory.axioms.Faithfulness(scenario),
    theory.axioms.Reinforcement(scenario),
    theory.axioms.Pareto(scenario),
    theory.axioms.Neutrality(scenario),
}

problem = CheckRule(corpus_small, plurality_rule)
problem.solve(strategy = "SAT")

True

Finally, we can automatically generate a rule that satisfies our corpus. Here, the problem `FindRule` is used.

In [11]:
from COMSOC.problems import FindRule

problem = FindRule(corpus)
rule = problem.solve(strategy = "SAT")

print(rule)

#############################
F(#1:a>c>b) ---> {'a'}
F(#1:a>b>c) ---> {'a'}
F(#1:c>a>b) ---> {'c'}
F(#1:b>c>a) ---> {'b'}
F(#1:b>a>c) ---> {'b'}
F(#1:c>b>a) ---> {'c'}
F(#1:b>c>a, #1:c>b>a) ---> {'b'}
F(#2:c>b>a) ---> {'c'}
F(#2:b>a>c) ---> {'b'}
F(#1:a>b>c, #1:b>a>c) ---> {'a'}
F(#1:a>c>b, #1:b>c>a) ---> {'b', 'c', 'a'}
F(#2:c>a>b) ---> {'c'}
F(#1:a>b>c, #1:a>c>b) ---> {'a'}
F(#1:c>a>b, #1:c>b>a) ---> {'c'}
F(#1:b>a>c, #1:c>b>a) ---> {'b'}
F(#2:a>b>c) ---> {'a'}
F(#1:a>b>c, #1:c>b>a) ---> {'b', 'c', 'a'}
F(#1:a>c>b, #1:c>a>b) ---> {'a'}
F(#1:b>a>c, #1:b>c>a) ---> {'b'}
F(#1:a>c>b, #1:c>b>a) ---> {'c'}
F(#2:a>c>b) ---> {'a'}
F(#2:b>c>a) ---> {'b'}
F(#1:a>b>c, #1:c>a>b) ---> {'a'}
F(#1:a>c>b, #1:b>a>c) ---> {'a'}
F(#1:b>c>a, #1:c>a>b) ---> {'c'}
F(#1:a>b>c, #1:b>c>a) ---> {'b'}
F(#1:b>a>c, #1:c>a>b) ---> {'b', 'c', 'a'}
#############################


Of course, we'd expect this rule to satisfy the smaller corpus of axioms as well:

In [11]:
CheckRule(corpus_small, rule).solve(strategy = "SAT")

True

## Automated Justification

The package can also be used to solve the problem of the automated justification of voting outcomes. Consider the following profile, outcome, and corpus of axioms.

In [13]:
nVoters = 2
alternatives = {"a", "b", "c"}

scenario = theory.Scenario(nVoters, alternatives)

profile = scenario.get_profile("a>b>c,b>a>c")
outcome = scenario.get_outcome("a,b")
corpus = {
    theory.axioms.Faithfulness(scenario),
    theory.axioms.Reinforcement(scenario),
    theory.axioms.Cancellation(scenario),
    theory.axioms.Pareto(scenario),
    theory.axioms.Neutrality(scenario),
    theory.axioms.PositiveResponsiveness(scenario)
}

We can define the problem as follows:

In [16]:
from COMSOC.problems import JustificationProblem

problem = JustificationProblem(profile, outcome, corpus)

To solve it, we can again call the `.solve()` method, which this time returns an iterator over all justifications. As additional parameters, we impose a maximum depth in the search of 3, we activate the heuristics, and we set the maximum number of justifications retrieved to 5. Among these, we return the smallest justification (in terms of number of instances). Furthermore, we add some heuristic axioms.

In [17]:
from COMSOC.just import Symmetry, QuasiTiedWinner, QuasiTiedLoser

shortest = None
# Note: ignoring nontriviality as I still have to implement the relevant code for Positive Responsiveness;
# will be added soon.
for justification in problem.solve(extract = "SAT", nontriviality = "ignore", depth = 0,\
                                  heuristics = True, maximum = 1000, derivedAxioms = (Symmetry(scenario),\
                                  QuasiTiedWinner(scenario), QuasiTiedLoser(scenario))):
    
    if shortest is None or len(justification) < len(shortest):
        shortest = justification
        
print(shortest)

ProcessLookupError: [Errno 3] No such process